# Traffic Sign Recognition Classifier


> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.

In [ ]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
from tensorflow import keras
import tensorflow.contrib.eager as tfe

import numpy as np

tf.enable_eager_execution()

print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

---

## The German Traffic Sign Recognition Benchmark [*](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset)

The below code assumes dataset files are placed in ./data folder and in specific subfolders of ./data

A convenience dataset download script `download_dataset.ssh` is included in this project directory, use this script to download the German Traffic Sign dataset.

---

## Step 0: Load The Data

`dataset` is a dictionary of dictionary with the following structure
<code style="background-color: #fff; font-size: 11px; line-height: 3px;">
    dataset = {
        'train' : {
            'X' : python-list of Image FileName Strings
            'y' : python-list of corresponding class name numbers
        }
        'val' : {
            'X' : python-list of Image FileName Strings
            'y' : python-list of corresponding class name numbers
        }
        'test' : {
            'X' : python-list of Image FileName Strings
            'y' : python-list of corresponding class name numbers
        }
    }
</code>

`train_dataset`, `val_dataset` and `test_dataset` are tf.data objects

`repeat()` is used so that tf.data iterator does not run out of training examples or so that iterator behaves like a circular list



### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
from utils import *

_BATCH_SIZE = 32
_NUM_CLASS = 43
_IMG_SHAPE = (32, 32, 3)

init_global_vars(_NUM_CLASS, _IMG_SHAPE, _BATCH_SIZE)

dataset = get_traffic_sign_dataset();

train_dataset = covert_to_tf_dataset(dataset["train"])
val_dataset = covert_to_tf_dataset(dataset["val"])
test_dataset =  covert_to_tf_dataset(dataset["test"])

# next(iter(train_dataset))

---

## Step 1: Dataset Summary & Exploration

In [ ]:
print("Number of training   examples =", len(dataset["train"]["X"]))
print("Number of validation examples =", len(dataset["val"]["X"]))
print("Number of testing    examples =", len(dataset["test"]["X"]))

print("\n")
print("Image data shape  =", _IMG_SHAPE)
print("Number of classes =", _NUM_CLASS)

In [ ]:
class_name_dict = get_class_name_dict()

for k, v in class_name_dict.items():
    print("{:03d}, {:s}".format(k, v))

### Visualization of the dataset

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path

%matplotlib inline

plt.rcParams["figure.figsize"] = [16, 9]

In [ ]:
# https://matplotlib.org/gallery/api/histogram_path.html#sphx-glr-gallery-api-histogram-path-py

def plot_histogram(labels, title):
    fig, ax = plt.subplots()
    n, bins = np.histogram(labels, _NUM_CLASS)
    
    # get the corners of the rectangles for the histogram
    left = np.array(bins[:-1])
    right = np.array(bins[1:])
    bottom = np.zeros(len(left))
    top = bottom + n
    
    # we need a (numrects x numsides x 2) numpy array for the path helper
    # function to build a compound path
    XY = np.array([[left, left, right, right], [bottom, top, top, bottom]]).T

    # get the Path object
    barpath = path.Path.make_compound_path_from_polys(XY)

    # make a patch out of it
    patch = patches.PathPatch(barpath)
    ax.add_patch(patch)

    # update the view limits
    ax.set_xlim(left[0], right[-1])
    ax.set_ylim(bottom.min(), top.max())
    
    plt.xlabel('Class Number')
    plt.ylabel('Class Count')
    plt.title(title)
    plt.grid(True)
    plt.show()


In [ ]:
plot_histogram(dataset["train"]["y"], 'Frequency of Class Examples in training dataset')

In [ ]:
plot_histogram(dataset["val"]["y"], 'Frequency of Class Examples in validation dataset')

----

## Step 2: LeNet-5 Neural network architecture

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). 

It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Model Architecture

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Input, Dropout
from tensorflow.keras.initializers import TruncatedNormal

μ = 0.0
σ = 0.05

'''
:::: Defaults ::::
      Conv2D: padding='valid', strides=(1, 1), use_bias=True, bias_initializer='zeros'
MaxPooling2D: pool_size=(2, 2), strides=None, padding='valid'
       Dense: use_bias=True, bias_initializer='zeros'
'''

model = Sequential()

# Input doesn't return a layer
# if you are using Sequential, you may also skip the Input
# and just specify the input_shape for the first Layer.
# model.add(Input(shape=(32, 32, 3)))

# In: (?, 32, 32, 3) | Out: (?, 28, 28, 6)
model.add(Conv2D(6, (5, 5), activation='relu', kernel_initializer=TruncatedNormal(mean=μ, stddev=σ), name='conv1'))
# In: (?, 28, 28, 6) | Out: (?, 14, 14, 6)
model.add(MaxPooling2D(strides=(2, 2), name='conv1_max_p'))
model.add(Dropout(0.3))

# In: (?, 14, 14, 6) | Out: (?, 10, 10, 16)
model.add(Conv2D(16, (5, 5), activation='relu', kernel_initializer=TruncatedNormal(mean=μ, stddev=σ), name='conv2'))
# In: (?, 10, 10, 16) | Out: (?, 5, 5, 16)
model.add(MaxPooling2D(strides=(2, 2), name='conv2_max_p'))
model.add(Dropout(0.3))

# In: (?, 5, 5, 16) | Out: (?, 400) {400 = 5*5*16}
model.add(Flatten(name='flatten'))

# In: (?, 400) | Out: (?, 120)
model.add(Dense(120, activation='relu', kernel_initializer=TruncatedNormal(mean=μ, stddev=σ), name='fc1'))
model.add(Dropout(0.5))

# In: (?, 120) | Out: (?, 84)
model.add(Dense(84, activation='relu', kernel_initializer=TruncatedNormal(mean=μ, stddev=σ), name='fc2'))
model.add(Dropout(0.5))

# In: (?, 84) | Out: (?, num_classes)
model.add(Dense(_NUM_CLASS, activation='softmax', kernel_initializer=TruncatedNormal(mean=μ, stddev=σ), name='fc3'))

- categorical_crossentropy should be used when labels are one_hot_encoded
- sparse_categorical_crossentropy for numbered labels

In [ ]:
model.compile(optimizer=tf.train.AdamOptimizer(3e-4), loss='categorical_crossentropy', metrics=['accuracy'])

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing.
A low accuracy on the training and validation sets imply underfitting.
A high accuracy on the training set but low accuracy on the validation set implies overfitting.

#### model.fit() and model.evaluate()

when using tf.data as input make sure to include steps parameter

In [ ]:
import pickle
import os

_EPOCHS = 50
_LOAD_MODEL = True

_MODEL_FILE = "./model/checkpoints/model_chkpt"
_HISTORY_FILE = "./model/train_history/train_history_dict.p"

history = None

if not _LOAD_MODEL:
    historyObj = model.fit(
        train_dataset,
        steps_per_epoch = len(dataset["train"]["X"]) // _BATCH_SIZE,
        epochs = _EPOCHS,
        validation_data = val_dataset,
        validation_steps = len(dataset["val"]["X"]) // _BATCH_SIZE,
        verbose = 1
    )
    history = historyObj.history
    
    model.summary()
    
    os.system("spd-say 'Model Trained'")
    
    model.save_weights(_MODEL_FILE)
    print("Saved model weights to disk")
    
    with open(_HISTORY_FILE, 'wb') as f:
        pickle.dump(history, f)
    print("Training history saved")
    
else:
    model.load_weights(_MODEL_FILE)
    print("Model weights loaded from disk")

    with open(_HISTORY_FILE, 'rb') as f:
        history = pickle.load(f)
    print("Training history loaded from disk")


In [ ]:
epochs = np.arange(_EPOCHS)

plt.plot(epochs, history['val_acc'], 'b.-', label = "Validation Accuracy")
plt.plot(epochs, history['acc'], 'r.-', label = "Training Accuracy")
plt.title('Validation Accuracy vs Training Accuracy')
plt.xlabel('Epoch(s)')
plt.ylabel('Accuracy')
leg = plt.legend(loc = 'lower right')
leg.get_frame().set_alpha(0.5)
plt.grid(True)
plt.show()

In [ ]:
plt.plot(epochs, history['val_loss'], 'b.-', label = "Validation Loss")
plt.plot(epochs, history['loss'], 'r.-', label = "Training Loss")
plt.title('Validation Loss vs Training Loss')
plt.xlabel('Epoch(s)')
plt.ylabel('Loss')
leg = plt.legend(loc = 'upper right')
leg.get_frame().set_alpha(0.5)
plt.grid(True)
plt.show()

#### To plot Model Arch, the following packages need to be installed, pydot, pydot-ng & graphviz

`pip3 install pydot --user`

`pip3 install graphviz --user`

`pip3 install pydot-ng --user`

In [ ]:
# from tensorflow.keras.utils import plot_model
# plot_model(model, show_shapes=True, to_file='./model/model_arch.png')

# from IPython.display import SVG
# from tensorflow.keras.utils.vis_utils import model_to_dot
# SVG(model_to_dot(model).create(prog='dot', format='svg'))

---

## Step 3: Testing the Model

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset, steps = len(dataset["test"]["X"]) // _BATCH_SIZE)

print('\n')
print('Test set loss:', test_loss)
print('Test set accuracy:', test_acc)

To get more insight into how our model is working, we downloaded few pictures of German traffic signs from the web and used our model to predict the traffic sign type.

### Load Dataset

In [ ]:
example_db = get_examples_dataset()
example_tf_db = covert_to_tf_dataset(example_db, repeat = False)

### Predict the Sign Type for Each Image

In [ ]:
predictions = model.predict(example_tf_db, steps = _BATCH_SIZE)
print(predictions.shape)

### Analyze Performance

[`tf.nn.top_k`](https://www.tensorflow.org/api_docs/python/tf/nn/top_k)

In [ ]:
values, indices = tf.nn.top_k(predictions, k = 5, sorted = True)
values = values.numpy()
indices = indices.numpy()

In [ ]:
N, K = values.shape
correct_pred = 0

from IPython.core.display import display, HTML

htmlMarkup = "<table>"
htmlMarkup += "<tr>"
htmlMarkup += "<th>Class#</th>"
htmlMarkup += "<th>Image</th>"
htmlMarkup += "<th>Label</th>"
htmlMarkup += "<th>Model Prediction</th>"
htmlMarkup += "</tr>"

for n in np.arange(N):
    correct_pred = (correct_pred + 1) if indices[n][0] == example_db["y"][n] else correct_pred
    htmlMarkup += "<tr>"
    htmlMarkup += "<td>{}</td>".format(example_db["y"][n])
    htmlMarkup += "<td><img src='{}' height='128' width='128'/></td>".format(example_db["X"][n])
    htmlMarkup += "<td>{}</td>".format(class_name_dict[example_db["y"][n]])
    s = ""
    for k in np.arange(K):
        s += "<p>{:5.4f}, {}, ({})</p>".format(values[n][k], class_name_dict[indices[n][k]], indices[n][k])
    htmlMarkup += "<td>{}</td>".format(s)
    htmlMarkup += "</tr>"

htmlMarkup += "</table>"
display(HTML(htmlMarkup))

print("{}/{} correctly classfified with an accuracy of {:5.4f}".format(correct_pred, N, correct_pred/N))

---

## Step 4: Visualize the Neural Network's State with Test Images

In [ ]:
import math

"""
activation should be of shape (1, H, W, C)
plt_num: used to plot out multiple different weight feature map sets on the same block,
        just extend the plt number for each new feature map entry

"""

def visualize_conv_layer_out(activation, activation_min = -1, activation_max = -1, plt_num = 1):
    _, H, W, C = activation.shape
    num_cols = math.floor(84 / H)
    num_rows = math.ceil(H / num_cols)
    plt.figure(plt_num, figsize=(num_rows, num_cols))

    featuremaps = activation.shape[3]
    plt.figure(figsize = (20, int(num_rows * H / 4.5))) # (width, height)
    
    for c in np.arange(C):
        # sets the number of feature maps to show on each row and column
        plt.subplot(num_rows, num_cols, c + 1)
        plt.axis('off')
        # displays the feature map number
        plt.title('Feature Map ' + str(c))
        
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, c], interpolation="nearest", vmin = activation_min,
                       vmax = activation_max, cmap = "gray")
        
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, c], interpolation="nearest", vmax = activation_max, cmap="gray")
        
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, c], interpolation="nearest", vmin=activation_min, cmap="gray")
        
        else:
            plt.imshow(activation[0,:,:, c], interpolation="nearest", cmap="gray")
    
    plt.tight_layout()
    plt.show()

In [ ]:
# https://keras.io/getting-started/faq/#how-can-i-obtain-the-output-of-an-intermediate-layer

def get_intermediate_layer_output(layer_name, x_test):
    layer = model.get_layer(name = layer_name)
    intermediate_layer_model = keras.Model(inputs = model.input, outputs = layer.output)
    return intermediate_layer_model.predict(x_test)

In [ ]:
import copy
from skimage import io

def plot_convolutions_of_random_test_image():
    viz_db = copy.deepcopy(dataset["test"])
    num = np.random.randint(len(viz_db["y"]))
    viz_db["X"] = [viz_db["X"][num]]
    viz_db["y"] = [viz_db["y"][num]]
    
    plt.imshow(io.imread(viz_db["X"][0]))
    plt.show()
    x_test, y_test = next(iter(covert_to_tf_dataset(viz_db, repeat = False)))
    
    print("conv1 visualization")
    visualize_conv_layer_out(get_intermediate_layer_output("conv1", x_test))
    
    print("conv2 visualization")
    visualize_conv_layer_out(get_intermediate_layer_output("conv2", x_test))
    
    print("-" * 200)

#### Plotting Convolutions of 'n' random test images

In [ ]:
for _ in np.arange(5):
    plot_convolutions_of_random_test_image()